In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from joblib import dump
from xgboost import XGBRegressor
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.datasets import make_blobs
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

C:\Users\Muril\AppData\Local\Temp\ipykernel_17896\2986201397.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from joblib import dump
from sklearn.cluster import MiniBatchKMeans
import seaborn as sns
from warnings import simplefilter


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

df = pd.read_csv('DataSet\\games.csv')

df.columns = df.columns.str.strip()
df.dropna()
df.drop(
    ['id', 'created_at', 'white_id', 'black_id', 'increment_code', 'opening_eco'],
    axis = 1,
    inplace = True
)

le = LabelEncoder()

print("Comecou!!")

df['moves'] = df["moves"].str.split()
moves = df['moves']
i = 0

for sublist in moves:
    moves = sublist[0: 10: 2]
    for item in sublist:
        if item in df.loc[i, 'moves']:
            df.loc[i, item] = 1
    i += 2
    if i > 20056:
        print("Terminou!!")
        break
df = df.fillna((int)(0))

df.drop(
    [ 'moves'],
    axis = 1,
    inplace = True
)

df['victory_status'] = le.fit_transform(df['victory_status'])
df['opening_name'] = le.fit_transform(df['opening_name'])
df['winner'] = le.fit_transform(df['winner'])
print(le.classes_)

Y = df['winner']
X = df.drop(['winner', 'winner'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = MiniBatchKMeans(n_clusters=6,
                        random_state=5464,
                        batch_size=40,
                        n_init="auto")
model.fit(X_train, X_test)
dump(model, 'chess.pkl')

preds = model.predict(X)
df['cluster'] = preds

data = list(zip(preds, Y, df['d4'].values.tolist()))

print(data)

win = 0
draw = 0
lost = 0

for i in data:
    if i[2] == 1:
        if i[0] == 0:
            if i[1] == 2:
                win += 1
            elif i[1] == 1:
                draw += 1
            else:
                lost += 1

    

total = win + draw + lost
porcent = win / total
        
print(porcent)
print("Acabou!!")


Comecou!!
Terminou!!
['black' 'draw' 'white']
[(1, 2, 1.0), (1, 0, 0.0), (1, 2, 0.0), (1, 2, 0.0), (1, 2, 0.0), (1, 1, 0.0), (1, 2, 1.0), (1, 0, 0.0), (1, 0, 0.0), (1, 2, 0.0), (1, 2, 0.0), (1, 0, 0.0), (1, 0, 1.0), (1, 0, 0.0), (1, 2, 0.0), (1, 0, 0.0), (1, 0, 0.0), (1, 0, 0.0), (1, 0, 0.0), (1, 2, 0.0), (1, 0, 0.0), (1, 2, 0.0), (1, 2, 0.0), (1, 2, 0.0), (1, 0, 1.0), (1, 2, 0.0), (1, 2, 1.0), (1, 0, 0.0), (1, 0, 0.0), (1, 0, 0.0), (1, 0, 0.0), (1, 0, 0.0), (1, 2, 0.0), (1, 0, 0.0), (1, 2, 0.0), (1, 0, 0.0), (1, 2, 0.0), (1, 0, 0.0), (1, 2, 1.0), (1, 2, 0.0), (2, 1, 0.0), (2, 0, 0.0), (2, 2, 1.0), (2, 2, 0.0), (2, 2, 0.0), (2, 2, 0.0), (2, 0, 0.0), (2, 2, 0.0), (2, 2, 1.0), (2, 2, 0.0), (2, 0, 1.0), (2, 2, 0.0), (2, 2, 1.0), (2, 0, 0.0), (1, 2, 1.0), (1, 2, 0.0), (1, 2, 0.0), (1, 0, 0.0), (1, 0, 1.0), (1, 2, 0.0), (1, 0, 1.0), (1, 2, 0.0), (1, 0, 1.0), (1, 2, 0.0), (1, 0, 1.0), (1, 2, 0.0), (1, 2, 1.0), (1, 0, 0.0), (1, 2, 0.0), (1, 0, 0.0), (1, 0, 1.0), (1, 2, 0.0), (1, 0, 1.0), (1, 

In [45]:
print(list(model.cluster_centers_[0]))

[0.8536585365853658, 1414949213651.049, 45.365853658536594, 2.1463414634146343, 1486.6829268292684, 1429.3658536585367, 748.0975609756098, 3.2195121951219514, 0.3414634146341463, 0.3170731707317074, 0.12195121951219512, 0.07317073170731708, 0.07317073170731708, 0.0975609756097561, 0.0, 0.0, 0.3658536585365854, 0.0, 0.1951219512195122, 0.0, 0.0, 0.24390243902439024, 0.3170731707317074, 0.1951219512195122, 0.04878048780487805, 0.04878048780487805, 0.0, 0.0975609756097561, 0.04878048780487805, 0.0, 0.0, 0.3414634146341463, 0.0, 0.04878048780487805, 0.1707317073170732, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024390243902439025, 0.0, 0.04878048780487805, 0.04878048780487805, 0.0, 0.07317073170731708, 0.04878048780487805, 0.04878048780487805, 0.07317073170731708, 0.024390243902439025, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07317073170731708, 0.0, 0.07317073170731708, 0.0, 0.0975609756097561, 0.04878048780487805, 0.04878048780487805, 0.0, 0.0, 0.07317073170731708, 0.0, 0.0975609756097561, 0.0, 0.0, 0.0